In [31]:
import pandas as pd
import re
from pandas import json_normalize
from sklearn.cluster import KMeans
from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler
from matplotlib.lines import Line2D
import random
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.cluster import KMeans
from ipywidgets import interact, widgets, Button, Layout
from IPython.display import Image, display, HTML

In [2]:
# !pip install spotipy

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [4]:
secrets_file = open("secrets.txt","r")

In [5]:
string = secrets_file.read()

In [6]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [7]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [8]:
songs = pd.read_csv('songs.csv')

In [9]:
df_with_audiof = pd.read_csv('df_with_audiof.csv')

In [10]:
X = df_with_audiof[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                    'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [12]:
def get_cluster_from_imput(x):
    search_results = sp.search(q=x, limit=1)
    song_id = search_results['tracks']['items'][0]['id']
    audio_features = sp.audio_features(song_id)
    audio_features_df = json_normalize(audio_features)
    X = audio_features_df[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                    'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
    # print('X', X)
    X_scaled = scaler.transform(X)
    # print('X_scaled', X_scaled)
    cluster = kmeans.predict(X_scaled)
    return cluster[0]

In [13]:
def pick_one_random_song(x):
    filtered_df = df_with_audiof[df_with_audiof['clusters'] == x]
    random_row = filtered_df.sample(n=1, random_state=random.seed())
    return random_row

In [14]:
kmeans = KMeans(n_clusters=15, random_state=42)  
kmeans.fit(X_scaled)

/Users/v.adragna/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=15, random_state=42)

In [15]:
get_cluster_from_imput('When I Was Your Man')

1

In [16]:
from ipywidgets import widgets

In [17]:
submit_button = widgets.Button(description="Submit")

In [50]:
songs.head()

,Unnamed: 0,title,artist
0,0,last time i saw you (sped up),Nicki Minaj
1,1,rich men north of richmond,Oliver Anthony Music
2,2,bongos,Cardi B & Megan Thee Stallion
3,3,slow dancing,V
4,4,slow dancing (piano ver.),V


In [55]:
from ipywidgets import interact, widgets
from IPython.display import Image, display
from ipywidgets import widgets

max_characters = 100
pd.set_option("display.max_colwidth", max_characters)

found = False

def handle_user_input(user_input):
    user_input = user_input.lower()
    
    if user_input in songs['title'].values:
        suggested_titles = songs[songs['title'] != user_input]['title']
        if not suggested_titles.empty:
            print('You like ' + user_input + ' from ' + songs[songs['title'] == user_input]['artist'] + " ?")
            print("You like hit songs. You might also like: " + suggested_titles.sample().values[0])
    else:
        search_results = sp.search(q=user_input, limit=10)
        song_info = search_results['tracks']['items'][0]['artists'][0]['name']
        print("I am going to search similar songs to " + user_input + " from " + search_results['tracks']['items'][0]['artists'][0]['name'])
        imput_cluster = get_cluster_from_imput(user_input)
        random_row = pick_one_random_song(imput_cluster)
        print('You might also like ' + random_row['track.name'] + ' by ' + random_row['name'])
        spotify_track_url = 'https://open.spotify.com/track/' + random_row['song_id']
        print("link: ", spotify_track_url)
        artist_info = sp.search(q=random_row['name'], type='artist', limit=1)
        image_url = artist_info['artists']['items'][0]['images'][0]['url']
        display(Image(url=image_url))

    text_input.value = ""

    
text_input = widgets.Text(description="Search")
text_input.layout.width = '800px' 



submit_button = widgets.Button(description="Submit")

def on_submit_button_click(b):
    user_input = text_input.value
    handle_user_input(user_input)

submit_button.on_click(on_submit_button_click)
text_input.on_submit(on_submit_button_click)

display(text_input)
display(submit_button)



Text(value='', description='Search', layout=Layout(width='800px'))

Button(description='Submit', style=ButtonStyle())

In [62]:
from ipywidgets import interact, widgets
from IPython.display import Image, display
from ipywidgets import widgets

max_characters = 100
pd.set_option("display.max_colwidth", max_characters)

found = False

def handle_user_input(user_input):
    user_input = user_input.lower()
    
    if user_input in songs['title'].values:
        suggested_titles = songs[songs['title'] != user_input]['title']
        if not suggested_titles.empty:
            print('You like ' + user_input + ' from ' + songs[songs['title'] == user_input]['artist'] + " ?")
            print("You like hit songs. You might also like: " + suggested_titles.sample().values[0])
    else:
        search_results = sp.search(q=user_input, limit=10)
        song_info = search_results['tracks']['items'][0]['artists'][0]['name']
        print("I am going to search similar songs to " + user_input + " from " + search_results['tracks']['items'][0]['artists'][0]['name'])
        imput_cluster = get_cluster_from_imput(user_input)
        random_row = pick_one_random_song(imput_cluster)
        print('You might also like ' + random_row['track.name'] + ' by ' + random_row['name'])
        spotify_track_url = 'https://open.spotify.com/track/' + random_row['song_id']
        print("link: ", spotify_track_url)
        artist_info = sp.search(q=random_row['name'], type='artist', limit=1)
        image_url = artist_info['artists']['items'][0]['images'][0]['url']
        display(Image(url=image_url))

    text_input.value = ""

    
text_input = widgets.Text(description="Search")
text_input.layout.width = '600px' 

submit_button = widgets.Button(description="Submit")

def on_submit_button_click(b):
    user_input = text_input.value
    handle_user_input(user_input)

submit_button.on_click(on_submit_button_click)
text_input.on_submit(on_submit_button_click)

display(text_input)
display(submit_button)

Text(value='', description='Search', layout=Layout(width='600px'))

Button(description='Submit', style=ButtonStyle())